In [60]:
# Step 1
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
path_to_flow_data = ""
df = pd.read_csv(path_to_flow_data) 

In [63]:
start_time = time.time()

In [64]:
df

,flow_id,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,0,0,0,0,0,0.0,2,0.0,46.0,0.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0.0
1,0,0,0,0,0,0.0,2,0.0,46.0,0.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0.0
2,0,0,0,0,0,0.0,2,0.0,46.0,0.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0.0
3,0,0,0,0,0,0.0,2,0.0,46.0,0.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0.0
4,0,0,0,0,0,0.0,2,0.0,46.0,0.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113083,3521,29,35811,36,23396,NaN,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
113084,3522,29,35811,36,23396,NaN,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
113085,3523,29,35811,36,23396,NaN,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
113086,3524,29,35811,36,23396,NaN,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN


In [65]:
df.drop(columns=['label'], inplace=True)
df.rename(columns={'attack_cat': 'label'}, inplace=True)

In [66]:
df.label.value_counts()

7    63441
6    40465
5     5612
3     1484
4     1058
2      424
1      290
0      259
8       50
9        5
Name: label, dtype: int64

In [69]:
# Z-score normalization

df['label'] = df['label'].astype('object')

features = df.dtypes[df.dtypes != 'object'].index
df[features] = df[features].apply(
    lambda x: (x - x.mean()) / (x.std()))

# Fill empty values by 0
df = df.fillna(0)

In [70]:
labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])

In [76]:
# retain the minority class instances and sample the majority class instances

df_minor = df[(df['label']==9)|(df['label']==8)|(df['label']==0)|(df['label']==1)|(df['label']==2)|(df['label']==4)|(df['label']==3)|(df['label']==5)]
df_major = df.drop(df_minor.index)

In [77]:
X = df_major.drop(['label'],axis=1) 
y = df_major.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [79]:
# use k-means to cluster the data samples and select a proportion of data from each cluster
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0).fit(X)

In [80]:
klabel=kmeans.labels_
df_major['klabel']=klabel

In [81]:
df_major['klabel'].value_counts()

100    637
149    517
589    483
370    447
970    342
      ... 
838      9
560      8
305      8
892      7
818      2
Name: klabel, Length: 997, dtype: int64

In [82]:
cols = list(df_major)
cols.insert(78, cols.pop(cols.index('label')))
df_major = df_major.loc[:, cols]

In [83]:
df_major

,flow_id,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,klabel,label
0,-1.471246,-2.956741,-1.629957,-1.940842,-1.757805,-0.567395,-0.175499,-0.059054,-0.094576,-0.279417,...,-0.166673,-0.63157,-0.604799,-0.42132,-0.478922,-0.12561,-0.059511,-0.237942,64,7
1,-1.471246,-2.956741,-1.629957,-1.940842,-1.757805,-0.567395,-0.175499,-0.059054,-0.094576,-0.279417,...,-0.166673,-0.63157,-0.604799,-0.42132,-0.478922,-0.12561,-0.059511,-0.237942,981,7
2,-1.471246,-2.956741,-1.629957,-1.940842,-1.757805,-0.567395,-0.175499,-0.059054,-0.094576,-0.279417,...,-0.166673,-0.63157,-0.604799,-0.42132,-0.478922,-0.12561,-0.059511,-0.237942,981,7
3,-1.471246,-2.956741,-1.629957,-1.940842,-1.757805,-0.567395,-0.175499,-0.059054,-0.094576,-0.279417,...,-0.166673,-0.63157,-0.604799,-0.42132,-0.478922,-0.12561,-0.059511,-0.237942,981,7
4,-1.471246,-2.956741,-1.629957,-1.940842,-1.757805,-0.567395,-0.175499,-0.059054,-0.094576,-0.279417,...,-0.166673,-0.63157,-0.604799,-0.42132,-0.478922,-0.12561,-0.059511,-0.237942,981,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113083,-1.346226,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,75,6
113084,-1.346190,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,75,6
113085,-1.346155,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,75,6
113086,-1.346119,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,75,6


In [84]:
def typicalSampling(group):
    name = group.name
    frac = 0.1
    return group.sample(frac=frac)

result = df_major.groupby(
    'klabel', group_keys=False
).apply(typicalSampling)

In [85]:
result['label'].value_counts()

7    6342
6    4048
Name: label, dtype: int64

In [86]:
result

,flow_id,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,klabel,label
85695,-1.041790,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,6
80378,-1.044133,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,6
80384,-1.043920,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,6
80379,-1.044098,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,6
80419,-1.042749,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56137,1.482964,0.425055,0.877858,-0.476975,-0.036552,-0.305177,-0.895813,-0.058634,-0.058594,-0.103134,...,-0.166673,-0.509132,-0.358084,-0.612190,-0.290476,-0.312436,-0.334091,-0.359482,999,7
49316,1.117242,0.164917,0.591775,-0.476975,0.572220,-0.305177,-0.895813,-0.058629,-0.059828,-0.103134,...,-0.166673,0.960120,0.628777,0.342158,-0.478922,-0.312436,-0.334091,-0.359482,999,7
54623,1.280716,0.294986,0.418414,-0.476975,0.651044,-0.305177,-0.895813,-0.058696,-0.064955,-0.143573,...,-0.166673,-0.141819,-0.111369,-0.421320,-0.102030,-0.312436,-0.334091,-0.359482,999,7
61846,1.664084,0.555124,0.939083,-0.563085,0.813801,-0.305177,-0.895813,-0.058874,-0.068657,-0.198273,...,-0.166673,-0.264257,-0.604799,-0.421320,-0.478922,-0.312436,-0.334091,-0.359482,999,7


In [87]:
result = result.drop(['klabel'],axis=1)
result = result.append(df_minor)

In [88]:
result.to_csv('./UNSW_final_baseline_undersampled.csv',index=0)

In [89]:
# Read the sampled dataset
df=pd.read_csv('./UNSW_final_baseline_undersampled.csv')  # was './CICIDS_final_baseline.csv'

In [90]:
df

,flow_id,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,...,ct_state_ttl,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,label
0,-1.041790,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
1,-1.044133,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
2,-1.043920,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
3,-1.044098,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
4,-1.042749,0.815262,0.989555,1.159112,0.958216,0.000000,1.265130,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19567,0.058036,-0.615498,0.844210,-0.907524,-1.218803,-0.305177,-0.895813,-0.052035,-0.090612,-0.276031,...,0.964813,-0.166673,-0.141819,-0.111369,-0.039581,0.086416,-0.125610,-0.059511,0.005140,4
19568,0.058107,-0.615498,0.845453,-0.907524,-1.218803,-0.305177,-0.895813,-0.031605,-0.088286,-0.275476,...,0.964813,-0.166673,-0.509132,-0.481442,-0.230451,-0.290476,-0.125610,-0.059511,-0.237942,4
19569,0.058107,-0.615498,0.845453,-0.907524,-1.218803,-0.305177,-0.895813,-0.036011,-0.087598,-0.275476,...,0.964813,-0.166673,-0.509132,-0.481442,-0.230451,-0.290476,-0.125610,-0.059511,-0.116401,4
19570,0.058178,-0.615498,0.848160,-1.251964,-1.708583,-0.305177,-0.895813,-0.036551,-0.083777,-0.232090,...,0.964813,-0.166673,-0.754007,-0.728157,-0.612190,-0.667368,-0.312436,-0.334091,-0.359482,3


In [91]:
df['label'] = df['label'].astype('object')
features = df.dtypes[df.dtypes != 'object'].index
df['label'] = df['label'].astype('int')

In [96]:
X = df.drop(['label'],axis=1).values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [100]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X_train, y_train)

In [101]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [102]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break        

In [103]:
# X_fs = subset_df[fs].values
X_fs = df[fs].values

In [104]:
X_fs.shape

(19572, 40)

In [105]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBFK(k = 20)

In [106]:
X_fss = fcbf.fit_transform(X_fs,y)

In [107]:
selected_features = fcbf.idx_sel

In [108]:
selected_features

[8, 6, 13, 10, 2, 19, 34, 7, 36, 37, 3, 1, 27, 5, 17, 4, 0, 35, 26, 25]

In [109]:
df['label'].value_counts()

7    6342
5    5612
6    4048
3    1484
4    1058
2     424
1     290
0     259
8      50
9       5
Name: label, dtype: int64

In [110]:
end_time = time.time()
print(f"Elapsed time: {end_time - start_time} seconds.")

Elapsed time: 17.99111008644104 seconds.
